In [ ]:
%load_ext dotenv
%dotenv

## Administrative boundaries

1. DJIBOUTI
2. ETHIOPIA
3. Eritrea
4. KENYA
5. SOMALIA
6. SOUTH SUDAN
7. SUDAN
8. UGANDA
9. BURUNDI
10. RWANDA
11. TANZANIA

Manually downloaded from the website and placed the files under, the unzip and shp files organization by following script

In [ ]:
mkdir TZA
mv gadm41_TZA_shp.zip TZA/
cd TZA 
unzip gadm41_TZA_shp.zip
cd ..


mkdir UGA
mv gadm41_UGA_shp.zip UGA/
cd UGA 
unzip gadm41_UGA_shp.zip
cd ..

mkdir SSD
mv gadm41_SSD_shp.zip SSD/
cd SSD
unzip gadm41_SSD_shp.zip
cd ..

mkdir SOM
mv gadm41_SOM_shp.zip SOM/
cd SOM
unzip gadm41_SOM_shp.zip
cd ..

mkdir ETH
mv gadm41_ETH_shp.zip ETH/
cd ETH
unzip gadm41_ETH_shp.zip
cd ..

mkdir RWA
mv gadm41_RWA_shp.zip RWA/
cd RWA
unzip gadm41_RWA_shp.zip
cd ..

mkdir KEN
mv gadm41_KEN_shp.zip KEN/
cd KEN 
unzip gadm41_KEN_shp.zip
cd ..

mkdir BDI
mv gadm41_BDI_shp.zip BDI/
cd BDI
unzip gadm41_BDI_shp.zip
cd ..

mkdir SDN
mv gadm41_SDN_shp.zip SDN/
cd SDN 
unzip gadm41_SDN_shp.zip
cd ..

mkdir ERI
mv gadm41_ERI_shp.zip ERI/
cd ERI 
unzip gadm41_ERI_shp.zip
cd ..

In [ ]:
import os
user_data_path=os.environ['data_path_administrative_boundary']

In [ ]:
import glob

import os
root=user_data_path

dirlist = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]

1. the below print can be opened in QGIS as copy paste to vector view 

In [ ]:
admin_boundary_filespath=[]
for dirs in dirlist:
    #print(f'"{user_data_path}{dirs}/gadm41_{dirs}_0.shp" ')
    admin_boundary_filespath.append(f'{user_data_path}{dirs}/gadm41_{dirs}_0.shp')

## Area of Interest and Region extent

In [ ]:
import geopandas as gp
import ntpath
import pandas as pd
from shapely.geometry import box

def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)




ed_list=[]
for admin_flpath in admin_boundary:
    db=gp.read_file(admin_flpath)
    cntry_code=path_leaf(admin_flpath)[:-4]
    #extent_dict=db.bounds.to_dict('list')
    extent_df=db.bounds
    extent_df['cntry_code']=cntry_code
    ed_list.append(extent_df)
    db=[]
    
    
edl=pd.concat(ed_list)
    
    
aoi_xmin=edl['minx'].min()
aoi_ymin=edl['miny'].min()
aoi_xmax=edl['maxx'].max()
aoi_ymax=edl['maxy'].max()

geometry = [box(aoi_xmin, aoi_ymin,aoi_xmax, aoi_ymax)]

er=pd.DataFrame()
er['name']=['east_africa_extent']

geodf = gp.GeoDataFrame(er, geometry=geometry)

#geodf.to_file(f'{user_data_path}ea_aoi_extent.shp')
geodf['geometry'].bounds

## Grid creation

In [ ]:
#!/usr/bin/python

import argparse
import pandas as pd
from shapely.geometry import Polygon, mapping
import simplekml
import numpy as np
import itertools
import geopandas as gp
import ntpath
import math

import shapely.wkt

# Instantiate the parser
"""
python multiple_gridkml_creator_v1.py gridextent_coords.csv

gridextent_coords.csv is input file having cordinate extents


"""

def map_extent(plot_extent): 
    geo_fe= Polygon([[plot_extent['w'],plot_extent['s']], [plot_extent['w'],plot_extent['n']], [plot_extent['e'],plot_extent['n']],[plot_extent['e'],plot_extent['s']]])
    return geo_fe

def crs_find_centroid(swlong,swlat,gridx,gridy,nx,ny):
    nelong=swlong+(gridx*nx)
    nelat=swlat+(gridy*ny)
    plot_extent={}
    plot_extent['w'],plot_extent['s'],plot_extent['n'],plot_extent['e']=swlong,swlat,nelat,nelong
    geo_extent=map_extent(plot_extent)
    cen_map_extent=geo_extent.centroid
    lat=cen_map_extent.y
    lon=cen_map_extent.x
    utm_band = str((math.floor((lon + 180) / 6 ) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = '0'+utm_band
    if lat >= 0:
        epsg_code = '326' + utm_band
    else:
        epsg_code = '327' + utm_band
    return epsg_code


def gridcreator(swlong,swlat,gridx,gridy,nx,ny):
    """
    creates polygon grids as a geometry list for the speciifed variables

    Parameters
    ----------
    swlong : south west longitude value in degree decimals (float)
    swlat : south west latitude value in degree decimals (float)
    gridx : width size of the grid required in degress  (float)
    gridy : width size of the grid required in degress  (float)
    nx : number of grids required in x direction (integer)
    ny : number of grids required in y direction (integer)
            
    Returns
    -------
    a list of polygons geometry for the given variables
  
    Notes
    -------
    module for grid computation, effective to southern parts, 
    it really shrinking the grid size while moving to north
    """
    nelong=swlong+(gridx*nx)
    nelat=swlat+(gridy*ny)
    xmin,ymin,xmax,ymax= swlong, swlat, nelong, nelat
    gridWidth = gridx
    gridHeight = gridy
    rows = round((ymax-ymin)/gridHeight)
    cols = round((xmax-xmin)/gridWidth)
    ringXleftOrigin = xmin
    ringXrightOrigin = xmin + gridWidth
    ringYtopOrigin = ymax
    ringYbottomOrigin = ymax-gridHeight
    polygun1=[]
    for i in np.arange(cols):
        ringYtop = ringYtopOrigin
        ringYbottom =ringYbottomOrigin
        for j in np.arange(rows):
            polygon = Polygon([(ringXleftOrigin, ringYtop), (ringXrightOrigin, ringYtop), (ringXrightOrigin, ringYbottom), (ringXleftOrigin, ringYbottom)])
            polygun1.append(polygon)
            ringYtop = ringYtop - gridHeight
            ringYbottom = ringYbottom - gridHeight
        ringXleftOrigin = ringXleftOrigin + gridWidth
        ringXrightOrigin = ringXrightOrigin + gridWidth
    return polygun1   




def maille_counter(grid_df):
    """
    creates a pandas datframe with Maille and Maille_X and Maille_Y columns
    Parameters
    ----------
    grid_df : a pandas dataframe with polygon geometries
            
    Returns
    -------
    a pandas dataframe with Maille columns
    """
    grid_df.columns=['geometry']
    grid_df['gn']=np.arange(1,len(grid_df.index)+1,1)
    wef=grid_df['gn'].to_numpy()
    # it's a gymnastics to get sw count for grids
    o=wef.reshape(nx, ny)
    p=o.T
    q=np.ravel(p,order='F')
    b=wef.reshape(ny, nx)
    c=b.T
    d=np.rot90(c, 1)
    e=np.ravel(d,order='F')
    qq=pd.DataFrame()
    qq['gn']=q
    qq['Maille']=e
    qq1=qq.sort_values('Maille')
    ww1=pd.merge(grid_df,qq1)
    #this is for giving the column(Maille_X) and row(Maille_Y) numbering
    rows=np.arange(1,ny+1,1)
    cols=np.arange(1,nx+1,1)
    rows1=[rows] * nx
    rows2=[item for sublist in rows1 for item in sublist]
    rows3=np.fliplr([rows2])[0]
    cols1=list(itertools.chain.from_iterable(itertools.repeat(x,ny) for x in cols))
    ww1['Maille_Y']=rows3
    ww1['Maille_X']=cols1
    return ww1



def xy_maker(maille_grid,epsg_code):
    """
    creates a geopandas dataframe with centroid, lower end and upper end cordinate details
    calculates grid area

    Parameters
    ----------
    maille_grid : Dataframe with Maille grids
            
    Returns
    -------
    Geopandas dataframe
    """
    maille_grid['centroid']=maille_grid['geometry'].map(lambda x: x.centroid)
    maille_grid["X"] = maille_grid.centroid.map(lambda p: p.x)
    maille_grid["Y"] = maille_grid.centroid.map(lambda p: p.y)
    maille_grid['X1']=maille_grid['geometry'].map(lambda x: x.exterior.coords[3][0])
    maille_grid['Y1']=maille_grid['geometry'].map(lambda x: x.exterior.coords[3][1])
    maille_grid['X2']=maille_grid['geometry'].map(lambda x: x.exterior.coords[1][0])
    maille_grid['Y2']=maille_grid['geometry'].map(lambda x: x.exterior.coords[1][1])
    maille_grid.drop(['centroid','gn'], axis=1, inplace=True)
    maille_grid1=maille_grid.sort_values('Maille')
    maille_grid2=gp.GeoDataFrame(maille_grid1)
    maille_grid2.crs = {'init' :'epsg:4326'}
    maille_grid2['area_cell']=maille_grid2['geometry'].to_crs({'init': 'epsg:{0}'.format(epsg_code)})\
               .map(lambda p: p.area / 10**6)
    #maille_grid2['area_cell']=maille_grid2['geometry'].map(lambda p: p.area  / 1e+6 )
    maille_grid3=maille_grid2.reset_index()
    maille_grid3.drop(['index'], axis=1, inplace=True)
    return maille_grid3

def path_leaf(path):
    """
    Get the name of a file without any extension from given path

    Parameters
    ----------
    path : file full path with extension
    
    Returns
    -------
    str
       filename in the path without extension

    """
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

def shp_maker(ue_grid,filepath):
    """
    creates a kml file with onclick baloon details on Maille and cordinates

    Parameters
    ----------
    ue_grid: dataframe
    filepath: kml file name and path
            
    Returns
    -------
    null
    """
    shpfilename=filepath
    csvfilename=filepath.replace('.shp','.csv')
    float_formatcolumns=['X', 'Y', 'X1', 'Y1','X2', 'Y2', 'area_cell']
    #ue_grid=ue_grid0.round(3)
    #print(ue_grid.head())
    ue_grid_p1=ue_grid.drop(float_formatcolumns, axis=1)
    ue_grid_p2=ue_grid[float_formatcolumns].applymap(lambda x: '{0:.3f}'.format(x))
    ue_grid=ue_grid_p1.join(ue_grid_p2)
    ue_grid['geometry']= ue_grid['geometry'].apply(lambda x: shapely.wkt.loads(shapely.wkt.dumps(x, rounding_precision=3)))
    ue_grid.to_file(shpfilename,index=False)
    #ue_grid.to_file(shpfilename,index=False)
    ue_grid1=ue_grid.drop(['geometry'], axis=1)
    ue_grid2=ue_grid1[['Maille', 'Maille_X', 'Maille_Y', 'X1', 'Y1', 'X2', 'Y2', 'X', 'Y','area_cell']]
    ue_grid2.info()
    ue_grid2.to_csv(csvfilename,index=False, float_format='%.3f') 

### Different resolution and grid count sizes for the EA AOI

#### For 50km reolsution grid

In [ ]:
swlong=21.838949
swlat=-11.745695 
maxx=51.415695 
maxy=23.145147

gridx=0.5
gridy=0.5
nx=round((maxx-swlong)/gridx)
ny=round((maxy-swlat)/gridy)
print(nx,ny)
#epsg_code=crs_find_centroid(swlong,swlat,gridx,gridy,nx,ny)
epsg_code=crs_find_centroid(swlong,swlat,gridx,gridy,nx,ny)
print("EPSG code of the given region is ",epsg_code)
grids_poly=gridcreator(swlong,swlat,gridx,gridy,nx,ny)
grid_df=pd.DataFrame(grids_poly)
maille_grid_df=maille_counter(grid_df)
epsg_code=crs_find_centroid(swlong,swlat,gridx,gridy,nx,ny)
ue_grid=xy_maker(maille_grid_df,epsg_code)
filepath=f'{user_data_path}ea_50km_grids.shp'
shp_maker(ue_grid,filepath)


#### Various reosolutions for EA region and theirs comparision

In [ ]:
swlong=21.838949
swlat=-11.745695 
maxx=51.415695 
maxy=23.145147

gridxs=[0.5,0.25,0.1,0.05,0.01]
gridys=[0.5,0.25,0.1,0.05,0.01]

tg_list=[]
for gx, gy in zip(gridxs, gridys):
    nx=round((maxx-swlong)/gx)
    ny=round((maxy-swlat)/gy)
    total_grids=nx*ny
    tg_list.append(total_grids)
    

db=pd.DataFrame()
db['gridxs']=gridxs
db['approx_grids_resolution_km']=['50 km','25 km','10 km','5 km','1 km']
db['total_number_grids']=tg_list
db